In [20]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
%matplotlib tk
import pyxem as pxm
import pymatgen as pmg
import numpy as np
import hyperspy.api as hs

In [22]:
fp = '/home/pc494/Documents/tutorials/pycrystem_examples/nanowire_precession.hdf5'

Nanowire down (110) - nearest peaks are 111 then 002

In [23]:
dp = pxm.ElectronDiffraction(pxm.load(fp))
dp.apply_affine_transformation(np.array([[0.99,0,0],
                                         [0,0.69,0],
                                         [0,0,1]]));

A Jupyter Widget

In [24]:
#alpha = dp.get_direct_beam_position(sigma=5)

First you build your structure

In [25]:
Ga = pmg.Element("Ga")
As = pmg.Element("As")
lattice = pmg.Lattice.cubic(5.6535)

In [26]:
structure = pmg.Structure.from_spacegroup("F23",lattice, [Ga,As], [[0, 0, 0],[0.25,0.25,0.25]])

In [27]:
recip_d_111 = np.sqrt((3/5.6535**2))

Next you set up your microscope paramaters in order to generate the templates

In [28]:
edc = pxm.DiffractionGenerator(300, 5e-2)
diff_gen = pxm.DiffractionLibraryGenerator(edc)

In [29]:
recip_d_111/11.4

0.026874375407392053

Now you specify what is going into the simulation microscope, in terms of phase and oreintation

In [30]:
from pyxem.utils.sim_utils import equispaced_so3_grid

In [31]:
rot_list = equispaced_so3_grid(90,180,90,5)
struc_lib = dict()
struc_lib["A"] = (structure,rot_list)
library = diff_gen.get_diffraction_library(struc_lib,
                                            calibration=recip_d_111/11.4,
                                            reciprocal_radius=0.75,
                                            half_shape=(72,72),
                                            representation='euler',
                                            with_direct_beam=False)

In [32]:
dp = dp.inav[17:27,35:45]
dp.set_calibration((recip_d_111/11.4))

With your library built you are ready to tackle the experimental data

In [33]:
from pyxem.indexation_generator import IndexationGenerator
indexer = IndexationGenerator(dp, library)
match_results = indexer.correlate()

A Jupyter Widget

In [34]:
from pyxem.utils.sim_utils import peaks_from_best_template
from pyxem.utils.plot import generate_marker_inputs_from_peaks

peaks= match_results.map(peaks_from_best_template,phase=["A"],library=library,inplace=False)

A Jupyter Widget

In [35]:
mmx,mmy = generate_marker_inputs_from_peaks(peaks)
dp.plot(cmap='viridis')
for mx,my in zip(mmx,mmy):
    m = hs.markers.point(x=mx,y=my,color='red',marker='x')
    dp.add_marker(m,plot_marker=True,permanent=True)

In [36]:
from matplotlib import pyplot as plt
#plt.close("all")